In [1]:
import math
import numpy as np

# Elliptical, parabolic, and hyperbolic orbits using the Euler method

In the simple_two_body.ipynb notebook I implemented circular orbits with arbitary starting positions and confirmed that the truncation error converged linearly with timestep for the Euler method.  

In this notebook I plan to implement hyperbolic, elliptical, and parabolic orbits using the Euler method. Parabolic orbits are marginally bound, hyperbolic orbits are unbound, and elliptical orbits are bound but oblong. All orbits should have the center of mass at one focus for both stars orbit and be symmetric between the two stars. 

Elliptical orbits should be elongated away from the other star. 

Hyperbolic and parabolic orbits should open away from the other star. 

All orbits should conserve energy and angular momentum. Later this could be enforced. I will probably not do that in this notebook. 

It would be meaningless to evaluate the L0 error for a hyperbolic or parabolic orbit since there is no cycle and the L0 error is found after one or more cycles when the system returns to its starting point. However, it would be possible to evaluate it for the elliptical orbit system, and I intend to check that the convergence of the L0 error with dt is linear as it should be for the Euler method. 

Because the convergence can be tested for ellipses only and because ellipses are closed so make it clear if they are not working correctly graphically, I will begin with ellipses then proceed to parabolas and hyperbolas. 

# Conic sections

Circles, ellipses, parabolas, and hyperbolas are called conic sections because they can all be obtained by taking slices of an infinite cone. 

Conic sections are related to each other through a parameter called eccentricity. Circles have an eccentricity of 0. Ellipses have an eccentricity of 0<e<1. Parabolas have an eccentricity of e=1. Hyperbolas have an eccentricity of e>1. 

Circles are described by the equation x^2+y^2=r^2 where r is the radius. They can also be described parametrically using sine and cosine. That is done below. 

In [2]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
radius=10
t=2*math.pi*0.01*np.arange(103)
centerx=3
centery=5
p = figure(title="Circle", plot_width=400, plot_height=400)
p.line(centerx+np.cos(t),centery+np.sin(t),line_width=3,line_color="blue",legend="Circle")
p.circle(centerx,centery,line_width=15,line_color="green",legend="Center")
p.legend.location="bottom_right"
show(p)

Loading BokehJS ...

Ellipses are shaped like ovals. They have two locations called focuses that determine their shapes. If a string were connected to both focuses, which are inside the ellipse, you could slide a pencil along the string and draw the ellipse keeping the string taut. The two focuses and the ellipse form a triangle with hypoteneuses of fixed length. 

Half the shorter of the two widths of the ellipse is called the semimajor axis, a. It's like a radius for ellipses. The lenght of the longer width is (1+e)*a. The focus is offset from the center by f=e*sqrt(1-e^2)*a. 

Ellipses can also be drawn at angles to the axis. An ellipse aligned with the x and y axis is shown below. 

The nonparametric equation for an ellipse is x^2/a^2+y^2/b^2=1
Below the ellipse is described parametrically. 

In [3]:
def getfocus(a,ecc):
    b=a*(1+ecc)
    #return np.sqrt(b**2-a**2)
    return ecc*np.sqrt(1-ecc**2)*b

In [4]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
radius=10
t=2*math.pi*0.01*np.arange(103)
a=10
ecc=.5
b=a*(1+ecc)
centerx=0
centery=0
focusx=centerx+getfocus(a,ecc)
focusy=centery+0.0
p = figure(title="Ellipse", plot_width=400, plot_height=400, x_range=[-15,15], y_range=[-15,15])
p.line(centerx+b*np.cos(t),centery+a*np.sin(t),line_width=3,line_color="blue",legend="Circle")
p.circle(focusx,focusy,line_width=15,line_color="green",legend="Focus 1")
p.circle(centerx-getfocus(a,ecc),focusy, line_width=15, line_color="red", legend="Focus 2")
p.legend.location="bottom_right"
show(p)

Loading BokehJS ...

In [5]:
import numpy as np
import matplotlib.pyplot as plt

def NewtonianForce(mass1,mass2, r):
    return (6.67408*10**-11)*mass1*mass2/r/r;

In [6]:
def euler(h,t, x,y,z,f,debugprint):
    xstep = (f(t, x, y,z)+f(t+h,x+h,y,z))/2*h
    xnew=x+xstep
    if(debugprint):
        print(h,t,x,xstep,xnew)
    return t+h,xnew

In [7]:
import math
def InitialDataEqualMassConic(radius,ecc,angle,initmass):
    orbitalangle=angle
    print("angle",angle)
    phi=np.array([math.pi+orbitalangle,orbitalangle])
    orbitalradius=radius*np.ones(2) #semimajor axis
    eccentricity=ecc
    print(eccentricity)
    mass=np.ones(2)
    masses=initmass*mass #*masssun (natural units)
    return phi,orbitalangle,orbitalradius,eccentricity, masses
    

In [8]:
import random,numpy as np
rad0=100
ecc0=0.5
angle0=0.0
mass0=1.0
initdateqellipse=InitialDataEqualMassConic(rad0,ecc0,angle0,mass0)
print(initdateqellipse)

angle 0.0
0.5
(array([3.14159265, 0.        ]), 0.0, array([100., 100.]), 0.5, array([1., 1.]))


In [10]:
def getxyuveqellipseredmass(initdat):
    phi,orbitangle,orbitalradius,eccentricity, masses=initdat
    print("initdat", orbitalradius, phi, np.cos(phi), np.sin(phi))
    metersperAU=1
    Gconstant=1
    #fix x0 y0 at one star, disregard initial data, use orbital radius as separation between stars
    #this is consistant with choice in previous part
    cosphi=np.cos(phi)
    sinphi=np.sin(phi)
    coordsep=orbitalradius #/2.
    print("coordsep",coordsep)
    #x0=orbitalradius/2.*np.cos(phi)*metersperAU
    #y0=orbitalradius/2.*np.sin(phi)*metersperAU
    count=0
    for phi0 in phi:
        if phi0==0:
            print("zero")
            #x0[count]=orbitalradius[count]/2.
            #y0[count]=0
            cosphi[count]=1.0
            sinphi[count]=0.0
        if phi0==math.pi:
            cosphi[count]=-1.0
            sinphi[count]=0.0
            print("pi")
            #x0[count]=-orbitalradius[count]/2.
            #y0[count]=0
        if phi0==math.pi/2.:
            cosphi[count]=0.0
            sinphi[count]=1.0
            #x0[count]=0
            #y0[count]=orbitalradius[count]/2.
        if phi0==3*math.pi/2.:
            cosphi[count]=0.0
            sinphi[count]=-1.0
            #x0[count]=0
            #y0[count]=-orbitalradius[count]/2.
        count+=1
    x0=coordsep*cosphi
    y0=coordsep*sinphi
    #x0[1]=0.0
    #y0[1]=0.0
    z0=np.zeros(2)
    print(x0)
    print(y0)
    v=np.zeros(2)
    a=np.zeros(2)
    ax0=0.
    ay0=0.
    az0=0.
    vx0=0.
    vy0=0.
    vz0=0.
    print(eccentricity)
    reducedmass=np.zeros(2)
    reducedmass[0]=masses[0]+masses[1]
    reducedmass[1]=masses[0]*masses[1]/reducedmass[0]
    #elliptical
    orbitalr=orbitalradius[0]/2. #reduced mass
    focusdisp=getfocus(orbitalr,eccentricity) #displacement of the center due to the focus being at the center
    E=-Gconstant*reducedmass[0]*reducedmass[1]/(2*orbitalr) #orbitalradius is correct here r1+r2.
    #This factor of half is due to the virial (VEE-REE-AHL) theorem, a stellar dynamics theorem governing the 
    #equipartion of degrees of freedom of energy in a system. Another way of writing it is T=-1/2*V. 2T-V=0
    coordsep=orbitalr*(1.+eccentricity) #starts at apastron
    starsep=coordsep #reduced mass, M is at center
    Eapastron=-Gconstant*reducedmass[0]*reducedmass[1]/(starsep) #virial theorem again 
    Fapastron=Eapastron/(starsep)
    print("E,F",E,Eapastron,2*Eapastron, 2*orbitalr,starsep)
    print("pos",coordsep, focusdisp, orbitalr)
    x0=(focusdisp+coordsep)*cosphi
    y0=(focusdisp+coordsep)*sinphi
    x0[0]=0.0
    y0[0]=0.0
    v=np.zeros(2)
    v[0]=0.0
    vwikipedia=np.sqrt(reducedmass[0]*(2./starsep-1./orbitalr))
    v[1]=np.sqrt(2.*(E-Eapastron)/reducedmass[1]) #1/2*mv^2-GMm/starsep=E. Solve for v. Eapastron=-GMm/2starsep
    print("vtest",vwikipedia,v[1])
    ux0=-v*sinphi
    uy0=v*cosphi #initial data in y only 
    uz0=np.zeros(2)
    
    a=np.zeros(2)
    a[0]=0.0
    a[1]=Fapastron/reducedmass[1]
    
    ax0=-a*cosphi
    ay0=-a*sinphi
    az0=np.zeros(2)

    
    
    return reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0

In [15]:
xyuvaeqellipseredmass=getxyuveqellipseredmass(initdateqellipse)
print(xyuvaeqellipseredmass)

initdat [100. 100.] [3.14159265 0.        ] [-1.  1.] [1.2246468e-16 0.0000000e+00]
coordsep [100. 100.]
pi
zero
[-100.  100.]
[0. 0.]
0.5
E,F -0.01 -0.013333333333333334 -0.02666666666666667 100.0 75.0
pos 75.0 32.47595264191645 50.0
vtest 0.11547005383792516 0.11547005383792516
(array([2. , 0.5]), array([  0.        , 107.47595264]), array([0., 0.]), array([0., 0.]), array([-0., -0.]), array([-0.        ,  0.11547005]), array([0., 0.]), array([0.        , 0.00035556]), array([-0.,  0.]), array([0., 0.]))


In [16]:
def timestep(step,t,dt,mtotal, mass,xi,yi,zi, vxi, vyi, vzi, axi, ayi, azi):
    xii=np.zeros(np.size(xi))
    vxii=np.zeros(np.size(vxi))
    yii=np.zeros(np.size(yi))
    vyii=np.zeros(np.size(vyi))
    zii=np.zeros(np.size(vzi))
    vzii=np.zeros(np.size(vzi))
    rii=np.zeros(np.size(xi))
    axii=np.zeros(np.size(axi))
    ayii=np.zeros(np.size(ayi))
    azii=np.zeros(np.size(azi))
    
    #This is redundant in the center of mass system
    Gconstant=6.408*10**-11
    Gconstant=1
    for k in np.arange(len(rii)):
        for j in np.arange(len(rii)):
            if j!=k:
                if j==0: #reduced mass
                    axii[0]=0.0
                    ayii[0]=0.0
                    azii[0]=0.0
                else:
                    rreljk=np.abs((xi[j] - xi[k])**2+(yi[j]-yi[k])**2+(zi[j]-zi[k])**2)**(1./2.)
                    #print(rreljk)
                    axii[j]-=Gconstant*mass[k]*(xi[j]  - xi[k])/rreljk**3
                    ayii[j]-=Gconstant*mass[k]*(yi[j]  - yi[k])/rreljk**3
                    azii[j]-=Gconstant*mass[k]*(zi[j]  - zi[k])/rreljk**3
    #print(xii)
    
    for m in np.arange(len(x)):
        #m represents choices of mass
        i=step
        
        xii[m] = xi[m] + dt*vxi[m]
        vxii[m] = vxi[m] + dt*axi[m]
        yii[m]= yi[m] + dt*vyi[m]
        vyii[m] = vyi[m] + dt*ayi[m]
        zii[m]= zi[m] + dt*vzi[m]
        vzii[m] = vzi[m] + dt*azi[m]
        rii[m]=np.sqrt(xi[m]**2+yi[m]**2+zi[m]**2)
        
        #print(vyii[m],vyi[m],ayi[m],dt)
        #print(yii[m],yi[m],vyi[m],dt)
    
    
    return mass, xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii
                    

In [17]:
dt=1 #*31556926 #seconds per year
numsteps=10000
mass0,x,y,z0,vx,vy,vz0,ax,ay,az0=xyuvaeqellipseredmass
print(x)
xcoord1=[]
xcoord2=[]
ycoord1=[]
ycoord2=[]
t=0.0
masstotal=1.
for i in np.arange(1,numsteps):
    mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(i,t,dt,masstotal,mass0,x,y,z0,vx,vy,vz0,ax,ay,az0)
    print(x,y,vx,vy,ax,ay)
    #print(ay) #forces should be equal and opposite, but in reduced mass framework accelerations are also equal and opposite
    #accelerations should evolve from y to x with time in a sinusoidal manner even in reduced mass framework
    #print(ax)
    xcoord1.append(x[0])
    xcoord2.append(x[1])
    ycoord1.append(y[0])
    ycoord2.append(y[1])
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(2,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(3,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(4,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 

[  0.         107.47595264]
[  0.         107.47595264] [0.         0.11547005] [0.         0.00035556] [-0.          0.11547005] [ 0.         -0.00017314] [0. 0.]
[  0.        107.4763082] [0.         0.23094011] [0.         0.00018241] [0.         0.11547005] [ 0.         -0.00017314] [ 0.00000000e+00 -1.86022168e-07]
[  0.         107.47649061] [0.         0.34641016] [0.00000000e+00 9.26790108e-06] [0.         0.11546987] [ 0.         -0.00017314] [ 0.00000000e+00 -3.72038712e-07]
[  0.         107.47649988] [0.         0.46188003] [ 0.         -0.00016387] [0.        0.1154695] [ 0.         -0.00017314] [ 0.00000000e+00 -5.58050395e-07]
[  0.       107.476336] [0.         0.57734953] [ 0.         -0.00033701] [0.         0.11546894] [ 0.         -0.00017314] [ 0.00000000e+00 -7.44057683e-07]
[  0.         107.47599899] [0.         0.69281846] [ 0.         -0.00051015] [0.         0.11546819] [ 0.         -0.00017314] [ 0.00000000e+00 -9.30061042e-07]
[  0.         107.47548884] [0

[  0.         104.20837261] [ 0.         22.62886513] [ 0.         -0.03366081] [0.         0.11185707] [ 0.         -0.00017188] [ 0.00000000e+00 -3.71283509e-05]
[  0.         104.17471181] [ 0.        22.7407222] [ 0.         -0.03383269] [0.         0.11181994] [ 0.         -0.00017187] [ 0.00000000e+00 -3.73223494e-05]
[  0.         104.14087912] [ 0.         22.85254215] [ 0.         -0.03400456] [0.         0.11178262] [ 0.         -0.00017186] [ 0.00000000e+00 -3.75164354e-05]
[  0.         104.10687455] [ 0.         22.96432477] [ 0.         -0.03417643] [0.        0.1117451] [ 0.         -0.00017185] [ 0.00000000e+00 -3.77106095e-05]
[  0.         104.07269813] [ 0.         23.07606987] [ 0.         -0.03434828] [0.         0.11170739] [ 0.         -0.00017184] [ 0.00000000e+00 -3.79048722e-05]
[  0.         104.03834985] [ 0.         23.18777727] [ 0.         -0.03452011] [0.         0.11166949] [ 0.         -0.00017183] [ 0.00000000e+00 -3.80992238e-05]
[  0.         104.00

[ 0.         99.94279729] [ 0.         33.70620474] [ 0.         -0.05095192] [0.        0.1071108] [ 0.         -0.00017037] [ 0.00000000e+00 -5.72471561e-05]
[ 0.         99.89184538] [ 0.         33.81331554] [ 0.         -0.05112229] [0.         0.10705356] [ 0.         -0.00017035] [ 0.00000000e+00 -5.74525207e-05]
[ 0.         99.84072309] [ 0.        33.9203691] [ 0.         -0.05129264] [0.        0.1069961] [ 0.         -0.00017033] [ 0.00000000e+00 -5.76580239e-05]
[ 0.         99.78943045] [ 0.        34.0273652] [ 0.         -0.05146298] [0.         0.10693845] [ 0.         -0.00017032] [ 0.00000000e+00 -5.78636663e-05]
[ 0.         99.73796747] [ 0.         34.13430365] [ 0.         -0.05163329] [0.         0.10688058] [ 0.        -0.0001703] [ 0.00000000e+00 -5.80694483e-05]
[ 0.         99.68633418] [ 0.         34.24118423] [ 0.         -0.05180359] [0.         0.10682251] [ 0.         -0.00017028] [ 0.00000000e+00 -5.82753705e-05]
[ 0.         99.63453059] [ 0.        

[ 0.         89.40460024] [ 0.         50.25950784] [ 0.        -0.0783959] [0.         0.09495466] [ 0.         -0.00016578] [ 0.00000000e+00 -9.29344101e-05]
[ 0.         89.32620433] [ 0.        50.3544625] [ 0.         -0.07856168] [0.         0.09486173] [ 0.         -0.00016574] [ 0.00000000e+00 -9.31695756e-05]
[ 0.         89.24764266] [ 0.         50.44932423] [ 0.         -0.07872741] [0.         0.09476856] [ 0.        -0.0001657] [ 0.00000000e+00 -9.34049704e-05]
[ 0.         89.16891525] [ 0.         50.54409279] [ 0.         -0.07889311] [0.         0.09467515] [ 0.         -0.00016566] [ 0.00000000e+00 -9.36405952e-05]
[ 0.         89.09002214] [ 0.         50.63876794] [ 0.         -0.07905876] [0.         0.09458151] [ 0.         -0.00016562] [ 0.00000000e+00 -9.38764506e-05]
[ 0.         89.01096337] [ 0.         50.73334945] [ 0.         -0.07922438] [0.         0.09448764] [ 0.         -0.00016557] [ 0.00000000e+00 -9.41125372e-05]
[ 0.       88.931739] [ 0.        

[ 0.         78.63018163] [ 0.         61.08645943] [ 0.         -0.09826747] [0.         0.08180796] [ 0.         -0.00015937] [ 0.         -0.00012349]
[ 0.         78.53191417] [ 0.         61.16826739] [ 0.         -0.09842684] [0.         0.08168446] [ 0.        -0.0001593] [ 0.         -0.00012376]
[ 0.         78.43348733] [ 0.         61.24995185] [ 0.         -0.09858614] [0.        0.0815607] [ 0.         -0.00015924] [ 0.         -0.00012403]
[ 0.         78.33490119] [ 0.         61.33151255] [ 0.         -0.09874538] [0.         0.08143667] [ 0.         -0.00015917] [ 0.        -0.0001243]
[ 0.        78.2361558] [ 0.         61.41294922] [ 0.         -0.09890455] [0.         0.08131238] [ 0.        -0.0001591] [ 0.         -0.00012457]
[ 0.         78.13725126] [ 0.        61.4942616] [ 0.         -0.09906365] [0.         0.08118781] [ 0.         -0.00015903] [ 0.         -0.00012483]
[ 0.         78.03818761] [ 0.         61.57544941] [ 0.         -0.09922268] [0.       

...

[  0.         -64.64337211] [ 0.         15.96809216] [ 0.         -0.03801714] [ 0.        -0.1942367] [0.         0.00043748] [ 0.         -0.00010944]
[  0.         -64.68138925] [ 0.         15.77385546] [ 0.         -0.03757967] [ 0.         -0.19434614] [0.         0.00043792] [ 0.         -0.00010817]
[  0.         -64.71896892] [ 0.         15.57950931] [ 0.         -0.03714174] [ 0.         -0.19445432] [0.         0.00043837] [ 0.        -0.0001069]
[  0.         -64.75611066] [ 0.         15.38505499] [ 0.         -0.03670338] [ 0.         -0.19456122] [0.        0.0004388] [ 0.         -0.00010563]
[  0.         -64.79281404] [ 0.         15.19049377] [ 0.         -0.03626458] [ 0.         -0.19466685] [0.         0.00043924] [ 0.         -0.00010436]
[  0.         -64.82907862] [ 0.         14.99582692] [ 0.         -0.03582534] [ 0.         -0.19477121] [0.         0.00043966] [ 0.         -0.00010308]
[  0.         -64.86490396] [ 0.         14.80105571] [ 0.         -0.

[  0.         -65.43886578] [  0.         -12.87560024] [0.       0.027681] [ 0.         -0.19610767] [0.         0.00044158] [0.00000000e+00 8.55244069e-05]
[  0.         -65.41118478] [  0.         -13.07170791] [0.         0.02812258] [ 0.         -0.19602215] [0.         0.00044118] [0.00000000e+00 8.68053454e-05]
[  0.        -65.3830622] [  0.         -13.26773006] [0.         0.02856376] [ 0.         -0.19593534] [0.         0.00044077] [0.00000000e+00 8.80839114e-05]
[  0.         -65.35449844] [  0.        -13.4636654] [0.         0.02900453] [ 0.         -0.19584726] [0.         0.00044036] [0.00000000e+00 8.93600754e-05]
[  0.         -65.32549391] [  0.         -13.65951266] [0.        0.0294449] [ 0.        -0.1957579] [0.         0.00043995] [0.00000000e+00 9.06338085e-05]
[  0.         -65.29604901] [  0.         -13.85527056] [0.         0.02988485] [ 0.         -0.19566726] [0.         0.00043953] [0.00000000e+00 9.19050815e-05]
[  0.         -65.26616416] [  0.       

[  0.         -57.51876889] [  0.       -38.972018] [0.         0.08356109] [ 0.         -0.17310195] [0.         0.00034391] [0.         0.00023165]
[  0.        -57.4352078] [  0.         -39.14511994] [0.       0.083905] [ 0.        -0.1728703] [0.       0.000343] [0.        0.0002324]
[  0.        -57.3513028] [  0.         -39.31799024] [0.       0.084248] [ 0.        -0.1726379] [0.         0.00034208] [0.         0.00023315]
[  0.        -57.2670548] [  0.         -39.49062814] [0.         0.08459008] [ 0.         -0.17240475] [0.         0.00034117] [0.         0.00023389]
[  0.         -57.18246471] [  0.         -39.66303289] [0.         0.08493125] [ 0.         -0.17217086] [0.         0.00034025] [0.         0.00023463]
[  0.         -57.09753346] [  0.         -39.83520375] [0.        0.0852715] [ 0.         -0.17193623] [0.         0.00033933] [0.         0.00023537]
[  0.         -57.01226196] [  0.         -40.00713998] [0.         0.08561083] [ 0.         -0.17170086] 

[  0.         -44.11118791] [  0.         -59.31365831] [0.         0.12027742] [ 0.         -0.13823902] [0.         0.00021938] [0.        0.0002935]
[  0.         -43.99091049] [  0.         -59.45189733] [0.        0.1204968] [ 0.         -0.13794552] [0.         0.00021844] [0.         0.00029372]
[  0.         -43.87041368] [  0.         -59.58984285] [0.         0.12071524] [ 0.        -0.1376518] [0.         0.00021749] [0.         0.00029393]
[  0.         -43.74969845] [  0.         -59.72749465] [0.         0.12093273] [ 0.         -0.13735787] [0.         0.00021655] [0.         0.00029414]
[  0.         -43.62876572] [  0.         -59.86485252] [0.         0.12114928] [ 0.         -0.13706373] [0.         0.00021561] [0.         0.00029435]
[  0.         -43.50761644] [  0.         -60.00191625] [0.         0.12136489] [ 0.         -0.13676938] [0.         0.00021467] [0.         0.00029456]
[  0.         -43.38625155] [  0.         -60.13868563] [0.         0.12157956] [ 

[  0.         -21.37386089] [  0.         -78.43528042] [0.        0.1450162] [ 0.         -0.08781563] [0.00000000e+00 8.02416796e-05] [0.         0.00029215]
[  0.         -21.22884469] [  0.         -78.52309606] [0.         0.14509644] [ 0.         -0.08752348] [0.00000000e+00 7.95638407e-05] [0.         0.00029197]
[  0.         -21.08374825] [  0.         -78.61061954] [0.         0.14517601] [ 0.         -0.08723151] [0.00000000e+00 7.88878094e-05] [0.        0.0002918]
[  0.         -20.93857224] [  0.         -78.69785105] [0.        0.1452549] [ 0.         -0.08693971] [0.00000000e+00 7.82135849e-05] [0.         0.00029162]
[  0.         -20.79331734] [  0.         -78.78479076] [0.         0.14533311] [ 0.         -0.08664809] [0.00000000e+00 7.75411664e-05] [0.         0.00029144]
[  0.         -20.64798423] [  0.         -78.87143885] [0.         0.14541065] [ 0.         -0.08635665] [0.00000000e+00 7.68705531e-05] [0.         0.00029126]
[  0.         -20.50257358] [  0. 

In [19]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(title="dt=1, Euler Method, Reduced Mass, e=0.3", plot_width=400, plot_height=400, y_range=[-150,150],x_range=[-150,150])
t = np.linspace(0, 2*math.pi, 100)
focusx=getfocus(0.5*rad0,ecc0)
b=0.5*rad0*(1+ecc)
print(b)
print(focusx)
# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(xcoord1,ycoord1, size=15, line_color="purple",fill_color="purple", fill_alpha=0.5, legend="M")
p.circle(xcoord2,ycoord2, size=1, line_color="blue", fill_color="blue", fill_alpha=0.5,legend="m")
p.line(focusx+b*np.cos(t),rad0*0.5*np.sin(t), line_color="green", legend="Ideal ellipse")
p.legend.location = "bottom_left"
p.legend.click_policy="hide"

show(p) # show the results

Loading BokehJS ...

75.0
32.47595264191645


Ellipse with wrong eccentricity and minor transients. Mathematically, it's focusdisp+coordsep, I can see now. I also fixed the theoretical ellipse so that the factor of 2 in the radius of the reduced mass system is accounted for. 

The starting positition is now right but it looks like either the initial acceleration is too low or the initial velocity is too high. It seems to still be a circle and I just can't figure out why. The velocity matches that on wikipedia-- which doesn't prove it's right, but if we are both getting the same answer, it should be. The acceleration is so straight forward. The radius of the circle is greater than 75, which should be the semimajor axis, and I am confused. 

Its possible all my spiraling behavior is due to such transients. I should go back and check at some point. However, the current transients look reasonable. 

HOW IS IT EVEN POSSIBLE TO EVOLVE FROM APHELION TO PERHELION AROUND A CENTRAL FIXED MASS IN A CIRCLE? HOW? A circle just doesn't solve the differential equation with those initial and final conditions!

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from math import pi

u=1.     #x-position of the center
v=0.5    #y-position of the center
a=2.     #radius on the x-axis
b=1.5    #radius on the y-axis

t = np.linspace(0, 2*pi, 100)
plt.plot(focusx+65*np.cos(t) , 50*np.sin(t) )
plt.grid(color='lightgray',linestyle='--')
plt.show()